In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atrial_Fibrillation"
cohort = "GSE115574"

# Input paths
in_trait_dir = "../../input/GEO/Atrial_Fibrillation"
in_cohort_dir = "../../input/GEO/Atrial_Fibrillation/GSE115574"

# Output paths
out_data_file = "../../output/preprocess/Atrial_Fibrillation/GSE115574.csv"
out_gene_data_file = "../../output/preprocess/Atrial_Fibrillation/gene_data/GSE115574.csv"
out_clinical_data_file = "../../output/preprocess/Atrial_Fibrillation/clinical_data/GSE115574.csv"
json_path = "../../output/preprocess/Atrial_Fibrillation/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data from human left and right atrial tissues in patients with degenerative MR in SR and AFib."
!Series_summary	"We aimed to compare the gene expression profiles of patients with degenerative MR in SR and AFib. We used Affymetrix human gene expression microarrays for each atrium sample. We chose most homogenous groups to compare, cause of the noise overshadows in high-throughput analysis when investigating complex diseases."
!Series_overall_design	"Left and right atrial tissue samples were obtained from patients with chronic primary severe MR in permanent AFib (n=15) and sinus rhythm (n=15). Transcriptomic analysis and bioinformatics have been done on all atrial tissues. Independent datasets from GEO were included in the analysis to confirm our findings. Real-time qPCR used to validate microarray results. Atrial tissues investigated via transmission electron microscopy for ultrasutructural changes."
Sample Characteristics Dictionar

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains gene expression microarray data
is_gene_available = True

# 2.1 Data Availability
# Identify rows in the sample characteristics dictionary for trait, age, and gender
trait_row = 0  # "disease state" contains information about atrial fibrillation status
age_row = None  # Age information is not available in the sample characteristics
gender_row = None  # Gender information is not available in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values to binary (0 for sinus rhythm, 1 for atrial fibrillation)"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert based on disease state
    if 'atrial fibrillation' in value.lower():
        return 1
    elif 'sinus rhythm' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric format"""
    # This function is defined for completeness but won't be used since age_row is None
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    # This function is defined for completeness but won't be used since gender_row is None
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM3182680': [1.0], 'GSM3182681': [1.0], 'GSM3182682': [1.0], 'GSM3182683': [1.0], 'GSM3182684': [1.0], 'GSM3182685': [1.0], 'GSM3182686': [1.0], 'GSM3182687': [1.0], 'GSM3182688': [1.0], 'GSM3182689': [1.0], 'GSM3182690': [1.0], 'GSM3182691': [1.0], 'GSM3182692': [1.0], 'GSM3182693': [1.0], 'GSM3182694': [1.0], 'GSM3182695': [1.0], 'GSM3182696': [1.0], 'GSM3182697': [1.0], 'GSM3182698': [1.0], 'GSM3182699': [1.0], 'GSM3182700': [1.0], 'GSM3182701': [1.0], 'GSM3182702': [1.0], 'GSM3182703': [1.0], 'GSM3182704': [1.0], 'GSM3182705': [1.0], 'GSM3182706': [1.0], 'GSM3182707': [1.0], 'GSM3182708': [0.0], 'GSM3182709': [0.0], 'GSM3182710': [0.0], 'GSM3182711': [0.0], 'GSM3182712': [0.0], 'GSM3182713': [0.0], 'GSM3182714': [0.0], 'GSM3182715': [0.0], 'GSM3182716': [0.0], 'GSM3182717': [0.0], 'GSM3182718': [0.0], 'GSM3182719': [0.0], 'GSM3182720': [0.0], 'GSM3182721': [0.0], 'GSM3182722': [0.0], 'GSM3182723': [0.0], 'GSM3182724': [0.0], 'GSM3182725': [0.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atrial_Fibrillation/GSE115574/GSE115574_series_matrix.txt.gz


Gene data shape: (54675, 59)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining gene identifiers
# These identifiers (like '1007_s_at', '1053_at') are Affymetrix probe IDs from a microarray
# They are not standard human gene symbols and will require mapping to official gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

Column 'ID' contains values matching gene data ID pattern


Column 'GB_ACC' contains values matching gene data ID pattern


Column 'Target Description' contains values matching gene data ID pattern


Column 'Representative Public ID' contains values matching gene data ID pattern


...

Column 'Gene Symbol' contains values matching gene data ID pattern


Column 'ENTREZ_GENE_ID' contains values matching gene data ID pattern


Column 'RefSeq Transcript ID' contains values matching gene data ID pattern


Column 'Gene Ontology Biological Process' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Species Scientific Name' may contain gene-related information
Sample values: ['Homo sapiens', 'Homo sapiens', 'Homo sapiens']
Column 'Target Description' may contain gene-related information
Sample values: ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene"]
Column 'Gene Title' may contain gene-related information
Sample values: ['discoidin domain receptor tyrosine kinase 1 /// microRNA 4640', 'replication factor C (activator 1) 2, 40kDa', "heat shock 70kDa protein 6 (HSP70B')"]
Column 'Gene Symbol' may contain gene-related information
Sample values: ['DDR1 /// MIR4640', 'RFC2', 'HSPA6']
Column 'ENTREZ_GENE_ID' may contain gene-related information
Sample values: ['780 /// 100616237

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the columns for gene identifiers and gene symbols
# From the output, we can see that 'ID' column in gene_annotation contains the same identifiers as in gene_data
# The 'Gene Symbol' column contains the gene symbols

# 2. Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# Preview the number of genes found
print(f"Number of genes after mapping: {len(gene_data)}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (45782, 2)
First few rows of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Mapped gene expression data shape: (21278, 59)
First few rows of gene expression data:
          GSM3182680  GSM3182681  GSM3182682  GSM3182683  GSM3182684  \
Gene                                                                   
A1BG        4.926930    4.932586    4.606200    5.171341    4.944955   
A1BG-AS1    3.103542    3.398962    3.103393    3.291102    3.197841   
A1CF        5.365683    5.590272    5.579943    5.255341    5.753891   
A2M        13.884166   13.669718   13.626048   13.419765   13.931109   
A2M-AS1     6.634899    6.839370    5.564249    6.530572    6.551003   

          GSM3182685  GSM3182686  GSM3182687  GSM3182688  GSM3182689  ...  \
Gene                                                                  ...   
A1BG        5.105716    4.501734    4.363352    5.128749    4.810223  ...   
A1BG-AS1    3.411080    3.318383    3.211218    2.799630    3.149896  ...   
A1CF        5.480124    5.600056    5.636956    5.567145    5.652957  ...   
A2M        13.828675   

Gene expression data saved to ../../output/preprocess/Atrial_Fibrillation/gene_data/GSE115574.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (21278, 59)
Gene data shape after normalization: (19845, 59)


Normalized gene expression data saved to ../../output/preprocess/Atrial_Fibrillation/gene_data/GSE115574.csv
Original clinical data preview:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   

                                          GSM3182680  \
0  disease state: atrial fibrillation patient wit...   
1                       tissue: left atrium  - heart   

                                          GSM3182681  \
0  disease state: atrial fibrillation patient wit...   
1                      tissue: right atrium  - heart   

                                          GSM3182682  \
0  disease state: atrial fibrillation patient wit...   
1                       tissue: left atrium  - heart   

                                          GSM3182683  \
0  disease state: atrial fibrillation patient wit...   
1                      tissue: right atrium  - heart   

                                          GSM3182684  \
0  disease state: atrial

Linked data shape before processing: (59, 19846)
Linked data preview (first 5 rows, 5 columns):
            Atrial_Fibrillation      A1BG  A1BG-AS1      A1CF        A2M
GSM3182680                  1.0  4.926930  3.103542  5.365683  13.884166
GSM3182681                  1.0  4.932586  3.398962  5.590272  13.669718
GSM3182682                  1.0  4.606200  3.103393  5.579943  13.626048
GSM3182683                  1.0  5.171341  3.291102  5.255341  13.419765
GSM3182684                  1.0  4.944955  3.197841  5.753891  13.931109


Data shape after handling missing values: (59, 19846)
For the feature 'Atrial_Fibrillation', the least common label is '1.0' with 28 occurrences. This represents 47.46% of the dataset.
A new JSON file was created at: ../../output/preprocess/Atrial_Fibrillation/cohort_info.json


Linked data saved to ../../output/preprocess/Atrial_Fibrillation/GSE115574.csv
